In [1]:
import os
import json
import glob
import h5py
import numpy as np

In [2]:
outdir = '../data_prods/'
data_dir = '/home/ryan/Data'
subdirs = [os.path.join(data_dir, o) for o in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir,o))]

subdir_list = []
for subdir in subdirs:
    subdir_list += [os.path.join(subdir, o) for o in os.listdir(subdir) if os.path.isdir(os.path.join(subdir,o))]

print(subdir_list)

['/home/ryan/Data/Data0/.Trash-1000', '/home/ryan/Data/Data0/run_2cDM_L3N256_DM_powerm2m2_sigma10', '/home/ryan/Data/Data0/run_CDM_L3N128_HY_var6', '/home/ryan/Data/Data0/run_CDM_L3N128_HY_var2', '/home/ryan/Data/Data0/run_2cDM_L3N128_HY_power00_sigma1', '/home/ryan/Data/Data0/run_2cDM_L3N256_HY_power00_sigma100', '/home/ryan/Data/Data0/lost+found', '/home/ryan/Data/Data0/run_2cDM_L3N256_DM_power00_sigma0.1', '/home/ryan/Data/Data0/run_2cDM_L3N128_HY_power00_sigma0.1', '/home/ryan/Data/Data0/run_2cDM_L3N256_DM_powerm2m2_sigma0.1', '/home/ryan/Data/Data0/other', '/home/ryan/Data/Data0/run_2cDM_L3N256_HY_power00_sigma0.1', '/home/ryan/Data/Data0/run_2cDM_L3N128_HY_power00_sigma0.1_var2', '/home/ryan/Data/Data0/run_2cDM_L3N128_DM_power00_sigma0.1', '/home/ryan/Data/Data0/run_2cDM_L3N128_HY_power00_sigma0.1_var5', '/home/ryan/Data/Data0/run_2cDM_L3N256_DM_power00_sigma1', '/home/ryan/Data/Data0/run_2cDM_L3N256_DM_power00_sigma100', '/home/ryan/Data/Data0/run_2cDM_L3N128_HY_power00_sigma0.1

In [3]:

def parsePower(powerType):
    stripped = powerType[5:]
    negFlag = False
    powers = [] 
    for char in stripped:
        if char == 'm':
            negFlag = True
            continue
        if negFlag:
            powers.append(-int(char))
            negFlag = False
        else:
            powers.append(int(char))

    return powers

def parseSigma(sigmaType):
    return float(sigmaType[5:])


In [4]:
def get_mass_profile(f):

    Subhalo=f.get('Subhalo')
    SubhaloMass=np.array(Subhalo['SubhaloMass'])
    SubhaloMass=10**10*SubhaloMass #converting to solar masses

    a=min(SubhaloMass)/2
    Mass_Bins=[]
    i=1
    while a<max(SubhaloMass):
        Mass_Bins.append(a)
        a=np.power(2,i/2)*min(SubhaloMass)
        i=i+1

    i=0
    j=0
    N_halo=[]
    for mbin in Mass_Bins:
        N_halo.append( (SubhaloMass > mbin).sum() )
    return Mass_Bins, N_halo

def get_mvel_profile(f):

    Subhalo=f.get('Subhalo') #Subhalo is the group, SubhaloVMas is the dataset inside the group
    SubhaloVMax=np.array(Subhalo['SubhaloVmax']) #km/s

    #Take Absolute Values of the velocities

    MaxVel=np.absolute(SubhaloVMax)

    #Gotta Create a maximum velocity bin

    a=0.5*min(MaxVel)
    MaxVel_Bin=[]
    while a<max(MaxVel):
        MaxVel_Bin.append(a)
        a=np.power(a,1.01)+a/5

    #Doing the magic here

    N_halo=[]
    i=0
    k=0
    for vbin in MaxVel_Bin:
        N_halo.append( (MaxVel > vbin).sum() )
    return MaxVel_Bin, N_halo

In [5]:
for subdir in subdir_list:
    
    #run name is just folder name
    run_name = subdir.split('/')[-1]

    #reject folders that aren't runs
    if 'run' not in run_name:
        continue

    fpath = os.path.join(outdir, run_name)
    try:
        os.mkdir(fpath)
    except:
        print(f'{fpath} already exists!')
    
    #get info from filename
    splitted = run_name.split('_')

    dm_type = splitted[1]

    # boxPart = splitted[2] # read from snapshot

    baryon_type = splitted[3]


    run_dict = {
        "run name": run_name,
        "DM type": dm_type,
        "baryon_type": baryon_type
    }

    if dm_type == 'CDM':
        otherInfo = splitted[4:]
        run_dict['otherInfo'] = otherInfo
    elif dm_type == '2cDM':
        powerType = splitted[4]
        powerLaws = parsePower(powerType)
        sigmaType = splitted[5]
        sigma = parseSigma(sigmaType)
        otherInfo = splitted[6:]

        temp = {
            "powerLaws": powerLaws,
            "sigma0": sigma,
            "otherInfo": otherInfo
        }

        run_dict.update(temp)



    # read info from snapshots/fof 
    snaps = sorted(glob.glob(subdir + '/snap*'))
    fofs = sorted(glob.glob(subdir + '/fof*'))

    with h5py.File(snaps[0], 'r') as f:
        boxSize = int((f['Header'].attrs['BoxSize'])) # in kpc
        NPart = int(np.cbrt(f['Header'].attrs['NumPart_Total'][1]))
        mass_resolution = f['Header'].attrs['MassTable'][1] * 10**10 # in solar masses
        softening_length = boxSize / NPart / 29
        temp = {
            "BoxSize": boxSize,
            "NPart": NPart,
            "Mass Resolution": mass_resolution,
            "Softening Length": softening_length
        }

        run_dict.update(temp)


    redshifts = []
    for i, fof in enumerate(fofs):
        with h5py.File(fof, 'r') as f:
            
            redshifts.append( float("{:.2f}".format(f['Header'].attrs['Redshift'])) )

            try:
                mbins, mcount = get_mass_profile(f)
                tname = os.path.join(fpath, "mass_profile_{}.txt".format(i))
                np.savetxt(tname, (mbins, mcount))
                vbins, vcount = get_mvel_profile(f)
                tname = os.path.join(fpath, "vel_profile_{}.txt".format(i))
                np.savetxt(tname, (vbins, vcount))
            except KeyError:
                continue

    run_dict['redshifts'] = redshifts

    fname = os.path.join(fpath, "run_info.json")
    with open(fname, 'w') as outfile:
        json.dump(run_dict, outfile)

../data_prods/run_2cDM_L3N256_DM_powerm2m2_sigma10 already exists!
../data_prods/run_CDM_L3N128_HY_var6 already exists!
../data_prods/run_CDM_L3N128_HY_var2 already exists!
../data_prods/run_2cDM_L3N128_HY_power00_sigma1 already exists!
../data_prods/run_2cDM_L3N256_HY_power00_sigma100 already exists!
../data_prods/run_2cDM_L3N256_DM_power00_sigma0.1 already exists!
../data_prods/run_2cDM_L3N128_HY_power00_sigma0.1 already exists!
../data_prods/run_2cDM_L3N256_DM_powerm2m2_sigma0.1 already exists!
../data_prods/run_2cDM_L3N256_HY_power00_sigma0.1 already exists!
../data_prods/run_2cDM_L3N128_HY_power00_sigma0.1_var2 already exists!
../data_prods/run_2cDM_L3N128_DM_power00_sigma0.1 already exists!
../data_prods/run_2cDM_L3N128_HY_power00_sigma0.1_var5 already exists!
../data_prods/run_2cDM_L3N256_DM_power00_sigma1 already exists!
../data_prods/run_2cDM_L3N256_DM_power00_sigma100 already exists!
../data_prods/run_2cDM_L3N128_HY_power00_sigma0.1_10e-7 already exists!
../data_prods/run_2cD